In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`io.delta:delta-spark_2.13:3.1.0`

Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.13/0.14.0-RC8/almond-spark_2.13-0.14.0-RC8.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.13/3.5.0/spark-sql_2.13-3.5.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.13/3.5.0/spark-parent_2.13-3.5.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammonite-spark_2.13/0.14.0-RC8/ammonite-spark_2.13-0.14.0-RC8.pom
Downloaded https://repo1.maven.org/maven2/com/univocity/univocity-parsers/2.9.1/univocity-parsers-2.9.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/xbean/xbean-asm9-shaded/4.23/xbean-asm9-shaded-4.23.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.13/3.5.0/spark-catalyst_2.13-3.5.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.13.1/parquet-column-1.13.1.pom
Downloaded https://repo1.maven.org

import $ivy.$
import $ivy.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()

import spark.implicits._

val tablePath = "/tmp/delta-scala"


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/05/03 14:41:14 INFO SparkContext: Running Spark version 3.5.0
25/05/03 14:41:14 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/05/03 14:41:14 INFO SparkContext: Java version 17.0.14
25/05/03 14:41:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/03 14:41:14 INFO ResourceUtils: ==============================================================
25/05/03 14:41:14 INFO ResourceUtils: No custom resources configured for spark.driver.
25/05/03 14:41:14 INFO ResourceUtils: ==============================================================
25/05/03 14:41:14 INFO SparkContext: Submitted application: ScalaSpark
25/05/03 14:41:14 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , v

import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@45e2c14c
import spark.implicits._
tablePath: String = "/tmp/delta-scala"

In [3]:
val df = Seq(
  (8, "bat"),
  (64, "mouse"),
  (1, "horse")
).toDF("number", "word")

25/05/03 14:41:18 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/05/03 14:41:18 INFO SharedState: Warehouse path is 'file:/workspaces/code/notebooks/spark-warehouse'.
25/05/03 14:41:19 INFO CodeGenerator: Code generated in 158.318453 ms


df: org.apache.spark.sql.package.DataFrame = [number: int, word: string]

In [4]:
df.write
  .format("delta")
  .mode("overwrite")  // or append
  .save(tablePath)

25/05/03 14:41:23 INFO DelegatingLogStore: LogStore `LogStoreAdapter(io.delta.storage.HDFSLogStore)` is used for scheme `file`
25/05/03 14:41:23 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/05/03 14:41:23 INFO InitialSnapshot: [tableId=39c5bfd9-bc6e-48ee-974b-ecd591bddd90] Created snapshot InitialSnapshot(path=file:/tmp/delta-scala/_delta_log, version=-1, metadata=Metadata(8b8671c3-1354-452f-879c-4541abb53cf7,null,null,Format(parquet,Map()),null,List(),Map(),Some(1746283283073)), logSegment=LogSegment(file:/tmp/delta-scala/_delta_log,-1,List(),org.apache.spark.sql.delta.EmptyCheckpointProvider$@197a4e60,-1), checksumOpt=None)
25/05/03 14:41:23 INFO DeltaLog: No delta log found for the Delta table at file:/tmp/delta-scala/_delta_log
25/05/03 14:41:23 INFO InitialSnapshot: [tableId=8b8671c3-1354-452f-879c-4541abb53cf7] Created snapshot InitialSnapshot(path=file:/tmp/delta-scala/_delta_log, version=-1, metadata=Metadata(4836e660-fea1-4272-a3

In [5]:
spark.read.format("delta").load(tablePath).show()

25/05/03 14:41:30 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/05/03 14:41:30 INFO CodeGenerator: Code generated in 80.261226 ms
25/05/03 14:41:30 INFO CodeGenerator: Code generated in 19.617155 ms
25/05/03 14:41:30 INFO BlockManagerInfo: Removed broadcast_7_piece0 on 08b3b062e527:44791 in memory (size: 119.8 KiB, free: 2.2 GiB)
25/05/03 14:41:30 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/05/03 14:41:30 INFO DAGScheduler: Got job 6 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/05/03 14:41:30 INFO DAGScheduler: Final stage: ResultStage 9 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/05/03 14:41:30 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 8)
25/05/03 14:41:30 INFO DAGScheduler: Missing parents: List()
25/05/03 14:41:30 INFO DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[32] at $anonfun$recordDe

+------+-----+
|number| word|
+------+-----+
|    64|mouse|
|     1|horse|
|     8|  bat|
+------+-----+

